In [36]:
import pyogrio
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

geopackage_path = 'C:/Users/eleonore.kong/Documents/InSAR/SWI/SWI_Package_1969-2023.gpkg'

In [37]:
df = pyogrio.read_dataframe(geopackage_path, layer='data_swi')
df

,SWI_UNIF_MENS3,YEAR,MONTH,geometry
0,1.137,2014,1,POINT (641374.000 7106309.000)
1,1.120,2014,2,POINT (641374.000 7106309.000)
2,1.092,2014,3,POINT (641374.000 7106309.000)
3,1.009,2014,4,POINT (641374.000 7106309.000)
4,0.897,2014,5,POINT (641374.000 7106309.000)
...,...,...,...,...
1077715,0.147,2023,8,POINT (1215772.000 6046242.000)
1077716,0.065,2023,9,POINT (1215772.000 6046242.000)
1077717,0.098,2023,10,POINT (1215772.000 6046242.000)
1077718,0.381,2023,11,POINT (1215772.000 6046242.000)


In [41]:
years = [2016,2017,2018,2019,2020,2021]
years_df = []
for year in years:
    df_year = df[df['YEAR'] == year]
    df_year = df_year.sort_values(by=['geometry', 'SWI_UNIF_MENS3'], ascending=True)
    df_low = df_year.groupby('geometry').first()
    df_high = df_year.groupby('geometry').last()
    
    df_low_high = df_low.merge(df_high[['MONTH', 'SWI_UNIF_MENS3']], on='geometry', how='outer', suffixes=('_trough', '_peak'))
    df_low_high.reset_index(inplace=True)
    years_df.append(df_low_high)

gdf = pd.concat(years_df, axis=0)
gdf = gpd.GeoDataFrame(gdf, crs=2154)
gdf['geometry'] = gdf['geometry'].buffer(3996, cap_style='square')
pyogrio.write_dataframe(gdf, path=geopackage_path, layer='data_swi_peak_trough')